In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as optimization
from scipy.optimize import least_squares
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise

In [2]:
df1 = pd.read_csv('df_ant1_30.csv') # in each channel we have 96 data points per antenna
df2 = pd.read_csv('df_ant2_30.csv')
df3 = pd.read_csv('df_ant3_30.csv')

In [3]:
t = df1[df1.Channel >= 11]
t['omega'] = (t['Channel']-11)*2+ 2428.25
tless = df1[df1.Channel < 11]
tless['omega'] = (tless['Channel'])*2 + 2404.25
df1 = t.append(tless)

<ipython-input-3-3d26178556e0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['omega'] = (t['Channel']-11)*2+ 2428.25
<ipython-input-3-3d26178556e0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tless['omega'] = (tless['Channel'])*2 + 2404.25


In [4]:
t = df2[df2.Channel >= 11]
t['omega'] = (t['Channel']-11)*2+ 2428.25
tless = df2[df2.Channel < 11]
tless['omega'] = (tless['Channel'])*2 + 2404.25
df2 = t.append(tless)

<ipython-input-4-efa26ec36f06>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['omega'] = (t['Channel']-11)*2+ 2428.25
<ipython-input-4-efa26ec36f06>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tless['omega'] = (tless['Channel'])*2 + 2404.25


In [5]:
t = df3[df3.Channel >= 11]
t['omega'] = (t['Channel']-11)*2+ 2428.25
tless = df3[df3.Channel < 11]
tless['omega'] = (tless['Channel'])*2 + 2404.25
df3 = t.append(tless)

<ipython-input-5-c0ab099caf1f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['omega'] = (t['Channel']-11)*2+ 2428.25
<ipython-input-5-c0ab099caf1f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tless['omega'] = (tless['Channel'])*2 + 2404.25


In [6]:
X12 = (df1['I'] + 1j*df1['Q']) * (df2['I'] - 1j*df2['Q'])
X23 = (df2['I'] + 1j*df2['Q']) * (df3['I'] - 1j*df3['Q'])
X13 = (df1['I'] + 1j*df1['Q']) * (df3['I'] - 1j*df3['Q'])

In [7]:
d = {'channel': df1['Channel'],'omega':df1['omega'],'x12': X12, 'x23': X23, 'x13': X13, 'idx1':df1['sample_idx'], 'idx2':df2['sample_idx'], 'idx3':df3['sample_idx']}
data = pd.DataFrame(data=d)
data['phi12'] = np.angle(X12)* 180/(2*np.pi)
data['phi23'] = np.angle(X23)* 180/(2*np.pi)
data['phi13'] = np.angle(X13)* 180/(2*np.pi)
# data.to_csv('phi_30degree.csv', index = False)

#### Another kalman filter
Here the Q and R is changing for each part. in the loop! 
This is not necessary, beacuse after a while, this will stay the same. So, not required. Haha!!

In [8]:
sdata = data

In [9]:
f = KalmanFilter (dim_x=2, dim_z=1)

In [10]:
f.x = np.array([[0.],   # theta
                [30.]])  #omega
f.F = np.array([[1.,0],
                [1.,1.]])
f.H = np.array([[0,1]])
f.P *= 1000.
f.R = np.array([[2.5]]) # dimentional
# f.Q = Q_discrete_white_noise(dim=2, dt=1e-6, var=0.13)

In [11]:
z12 = sdata['phi12']
results = []
for i in range(len(z12)):
    f.Q = Q_discrete_white_noise(dim=2, dt=1e-6, var=1/np.abs(sdata['x12'][i]))
    f.predict()
    f.update(z12[i])
    results.append(f.x)
res_arr = np.asarray(results)
sdata['phi12_kf'] = res_arr[:,0] 
sdata['omega12_kf'] = res_arr[:,1] 

In [12]:
z23 = sdata['phi23']
results = []
for i in range(len(z23)):
    f.Q = Q_discrete_white_noise(dim=2, dt=1e-6, var=1/np.abs(sdata['x23'][i]))
    f.predict()
    f.update(z23[i])
    results.append(f.x)
res_arr = np.asarray(results)
sdata['phi23_kf'] = res_arr[:,0] 
sdata['omega23_kf'] = res_arr[:,1] 

In [13]:
z13 = sdata['phi13']
results = []
for i in range(len(z13)):
    f.Q = Q_discrete_white_noise(dim=2, dt=1e-6, var=1/np.abs(sdata['x13'][i]))
    f.predict()
    f.update(z13[i])
    results.append(f.x)
res_arr = np.asarray(results)
sdata['phi13_kf'] = res_arr[:,0] 
sdata['omega13_kf'] = res_arr[:,1] 

In [14]:
print(np.average(sdata['phi23']), np.average(sdata['phi23_kf']), np.average(sdata['omega23_kf']))


16.448798889914162 -0.004285453729224755 20.41377899889309


In [15]:
print(np.average(sdata['phi12']), np.average(sdata['phi12_kf']), np.average(sdata['omega12_kf']))

55.24085644253681 -0.007332041066017729 53.19697618165385


In [16]:
print( np.average(sdata['phi13']), np.average(sdata['phi13_kf']), np.average(sdata['omega13_kf']))

-63.00629061349499 -0.007192888235967268 -47.905442114878305


In [17]:
phi_channel = []
for i in range(0, len(data),96):
    phi_channel.append({"channel":data['channel'][i], 
                       "omega":data['omega'][i], 
                       "ave_phi12": np.average(sdata['phi12'][i:i+96]),
                       "ave_phi23": np.average(sdata['phi23'][i:i+96]), 
                       "ave_phi13": np.average(sdata['phi13'][i:i+96]),
                       "omega12": np.average(sdata['omega12_kf'][i:i+96]),
                       "omega23": np.average(sdata['omega23_kf'][i:i+96]),
                       "omega13": np.average(sdata['omega13_kf'][i:i+96])})
saverage_phi_perchannel = pd.DataFrame(phi_channel)

In [18]:
saverage_phi_perchannel['omega12_kf'] = ((2.9*1e10)/(3.5*2*np.pi)*(1/saverage_phi_perchannel['omega'])*10**-6*saverage_phi_perchannel['omega12'])
saverage_phi_perchannel['omega23_kf'] = ((2.9*1e10)/(3.5*2*np.pi)*(1/saverage_phi_perchannel['omega'])*10**-6*saverage_phi_perchannel['omega23'])
saverage_phi_perchannel['omega13_kf'] = ((2.9*1e10)/(2*3.5*2*np.pi)*(1/saverage_phi_perchannel['omega'])*10**-6*saverage_phi_perchannel['omega13'])
saverage_phi_perchannel['theta12'] = ((2.9*1e10)/(3.5*2*np.pi)*(1/saverage_phi_perchannel['omega'])*10**-6*saverage_phi_perchannel['ave_phi12']) # c/f * xx * 1/2pi * 1/d
saverage_phi_perchannel['theta23'] = ((2.9*1e10)/(3.5*2*np.pi)*(1/saverage_phi_perchannel['omega'])*10**-6*saverage_phi_perchannel['ave_phi23'])
saverage_phi_perchannel['theta13'] = ((2.9*1e10)/(2*3.5*2*np.pi)*(1/saverage_phi_perchannel['omega'])*10**-6*saverage_phi_perchannel['ave_phi13'])

In [19]:
print('\n theta (12, 23, 13)', np.average(saverage_phi_perchannel['theta12']), np.average(saverage_phi_perchannel['theta23']), np.average(saverage_phi_perchannel['theta13']),
      '\n theta_KF', np.average(saverage_phi_perchannel['omega12_kf']), np.average(saverage_phi_perchannel['omega23_kf']), np.average(saverage_phi_perchannel['omega13_kf']))   


 theta (12, 23, 13) 29.8158820344641 8.879952658156277 -17.014881515953626 
 theta_KF 28.738188384167557 11.025569028913592 -12.940909162283244
